In [2]:
import os 
from prompts import context, code_parser_template, FORMAT_INSTRUCTIONS_TEMPLATE, react_system_header_str, EXTRACT_TEMPLATE_STR

from dotenv import load_dotenv
load_dotenv()


OLLAMA_URL=os.getenv("OLLAMA_URL")
LLM_MODEL=os.getenv("LLM_MODEL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
DATAPATH=os.getenv("DATAPATH")
DATAPATH='/data'

print(OLLAMA_URL)
print(f"model: {LLM_MODEL}")
print(f"datapath {DATAPATH}")

http://localhost:11434
model: qwen2.5:1.5b
datapath /data


In [3]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model=LLM_MODEL, request_timeout=120.0, base_url=OLLAMA_URL, temperature=0)

In [12]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, PromptTemplate
documents = SimpleDirectoryReader(
    input_files=[f"{DATAPATH}/testdata.txt",f"{DATAPATH}/additionalinfo.txt"]
).load_data()

In [19]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter

Settings.embed_model = OllamaEmbedding(model_name=EMBEDDING_MODEL, base_url=OLLAMA_URL,embed_batch_size=100)
Settings.text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)
# ollama
Settings.llm = llm

index = VectorStoreIndex.from_documents(
documents,
#embed_model=ollama_embedding,
)

ResponseError: <!--# V4wV54PZ3qT3Qr3nn34P5qD3kjV4wV54PZ3qT3Qd-->
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3c.org/TR/1999/REC-html401-19991224/loose.dtd">
<html>
<head>
<meta name="description" content="Zscaler makes the internet safe for businesses by protecting their employees from malware, viruses, and other security threats.">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Internet Security by Zscaler</title>
<script language="JavaScript">var defLang = 'en_US'</script>
<!--ATOS-->
<style type="text/css">
body {
background-color:#e3e3e3;
font-family:Arial, sans-serif;
font-size:12px;
color:#4B4F54;
}
a {
cursor:pointer;
text-decoration:none;
color:#009dd0;
}
table {
margin-top:10px;
}
td table {
margin-top:0;
text-align:center;
}
img {
max-height:75px;
max-width:430px;
}
.pg {
position:absolute;
top:0;
bottom:0;
left:0;
right:0;
overflow-x:hidden;
white-

In [16]:
from llama_index.core.tools import QueryEngineTool

query_engine = index.as_query_engine(similarity_top_k=3)

knowledge_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="knowledge_tool",
    description="""A RAG engine with some basic facts persons. Ask natural-language questions about persons and their properties and relations.
              if the knowledge_tool has no relatied information, ignore the answer.
              """
)


# generate_kwargs parameters are taken from https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct
@lru_cache(maxsize=10)
def find_person(name: str, **kwargs):
    """
    provides information about known persons. including ther detail information like birthdate

    args:
        name
    """
    # Mock response; replace with real query logic
    person_data = {
        "anna gölding": {"birthdate": "October 24, 1734", "known_for": "Last witch executed in Switzerland.","object_id":"1234","relations":{"knows other person":"ron paul","organzation":"pilz mafia"}},
        "john doe": {"birthdate": "Unknown", "known_for": "Placeholder name for anonymous individuals."},
        "ron paul": {"birthdate": "May 1, 1928", "known_for": "Talking a lot."},
        "miranda meyers": {"birthdate": "Aug 11, 1998", "known_for": "Miranda verkauft gerne verdorbens Eis. Das Eis erhält sie illegal von Litauen, wo es mit Mäusemilch hergestellt wird."},
    }
    return person_data.get(name.lower(), "No information available for this person.")

find_person_tool = FunctionTool.from_defaults(
    fn=find_person,
    name="find_person",
)

@lru_cache(maxsize=10)
def find_organization(name: str, **kwargs):
    """
    provides information about known official and inofficial organzations.

    args:
        name
    """
    # Mock response; replace with real query logic
    org_data  = {
        "un": {"name": "United Nations", "description": "The Security Council has primary responsibility for the maintenance of international peace and security.","id":"200","relations":{""}},
        "pilz mafia": {"name": "Pilz Mafia", "description": "","id":"201","members":{"anna gölding","ron paul"}},
        "acme company": {"name":"acme company","description":"placeholder company"},
    }
    return org_data.get(name.lower(), "No information available for this organization.")

find_orgnization_tool = FunctionTool.from_defaults(
    fn=find_organization,
    name="find_organization",
)

NameError: name 'index' is not defined